In [3]:
import sqlite3 as db

In [4]:
conn = db.connect('example.db')

In [5]:
# Create a 'cursor' for executing commands
c = conn.cursor()

In [14]:
# If this is not the first time you run this cell, 
# you need to delete the existed "Students" table first

c.execute("DROP TABLE IF EXISTS Students")    
c.execute("DROP TABLE IF EXISTS Takes") 

# create a table named "Students" with 2 columns: "gtid" and "name".
# the type for column "gtid" is integer and for "name" is text. 
c.execute('CREATE TABLE Takes (gtid INTEGER, course TEXT, grade REAL)')
c.execute("CREATE TABLE Students (gtid INTEGER, name TEXT)")

In [15]:
c.execute("INSERT INTO Students VALUES (123, 'Vuduc')")
c.execute("INSERT INTO Students VALUES (456, 'Chau')")
c.execute("INSERT INTO Students VALUES (381, 'Bader')")
c.execute("INSERT INTO Students VALUES (991, 'Sokol')")

In [16]:
more_students = [(723, 'Rozga'),
                 (882, 'Zha'),
                 (401, 'Park'),
                 (377, 'Vetter'),
                 (904, 'Brown')]

# '?' question marks are placeholders for the two columns in Students table
c.executemany('INSERT INTO Students VALUES (?, ?)', more_students)
conn.commit()

In [17]:
takes_data = [
    (123, 'CSE 6040', 4.0),
    (123, 'ISYE 6644', 3.0),
    (123, 'MGMT 8803', 1.0),
    (991, 'CSE 6040', 4.0),
    (991, 'ISYE 6740', 4.0),
    (456, 'CSE 6040', 4.0),
    (456, 'CSE 6740', 2.0),
    (456, 'MGMT 8803', 3.0)
]
c.executemany('INSERT INTO Takes VALUES (?, ?, ?)', takes_data)

__Commitment issues__. The commands above modify the database. However, these are temporary modifications and aren't actually saved to the databases until you say so. (Aside: Why would you want such behavior?) The way to do that is to issue a commit operation from the connection object.

In [18]:
conn.commit()

In [19]:
# Displays the results of your code
c.execute('SELECT * FROM Takes')
results = c.fetchall()
print("Your results:", len(results), "\nThe entries of Takes:", results)

Your results: 8 
The entries of Takes: [(123, 'CSE 6040', 4.0), (123, 'ISYE 6644', 3.0), (123, 'MGMT 8803', 1.0), (991, 'CSE 6040', 4.0), (991, 'ISYE 6740', 4.0), (456, 'CSE 6040', 4.0), (456, 'CSE 6740', 2.0), (456, 'MGMT 8803', 3.0)]


In [20]:
# Close the database
conn.close()

### Lesson 1: Join queries
The main type of query that combines information from multiple tables is the join query. Recall from our discussion of tibbles these four types:

- INNER JOIN(A, B): Keep rows of A and B only where A and B match
- OUTER JOIN(A, B): Keep all rows of A and B, but merge matching rows and fill in missing values with some default (NaN in Pandas, NULL in SQL)
- LEFT JOIN(A, B): Keep all rows of A but only merge matches from B.
- RIGHT JOIN(A, B): Keep all rows of B but only merge matches from A.
If you are a visual person, see this page for illustrations of the different join types.

In SQL, you can use the WHERE clause of a SELECT statement to specify how to match rows from the tables being joined. For example, recall that the Takes table stores classes taken by each student. However, these classes are recorded by a student's GT ID. Suppose we want a report where we want each student's name rather than his/her ID. We can get the matching name from the Students table. Here is a query to accomplish this matching:

In [ ]:
conn = db.connect('example.db')
c = conn.cursor()

#### See all (name, course, grade) tuples

In [22]:
query = '''
        SELECT Students.name, Takes.course, Takes.grade
        FROM Students, Takes
        WHERE Students.gtid = Takes.gtid
'''

for match in c.execute(query): # Note this alternative idiom for iterating over query results
    print(match)

('Vuduc', 'CSE 6040', 4.0)
('Vuduc', 'ISYE 6644', 3.0)
('Vuduc', 'MGMT 8803', 1.0)
('Chau', 'CSE 6040', 4.0)
('Chau', 'CSE 6740', 2.0)
('Chau', 'MGMT 8803', 3.0)
('Sokol', 'CSE 6040', 4.0)
('Sokol', 'ISYE 6740', 4.0)


_example of "AND"_

In [23]:
query = '''
        SELECT Students.name, Takes.grade
        FROM Students, Takes
        WHERE Students.gtid = Takes.gtid AND Takes.course = 'CSE 6040'
'''
c.execute(query)
results1 = c.fetchall()
results1

[('Vuduc', 4.0), ('Sokol', 4.0), ('Chau', 4.0)]

#### LEFT JOIN

In [24]:
query = '''
        SELECT Students.name, Takes.grade
        FROM Students LEFT JOIN Takes ON
        Students.gtid = Takes.gtid
'''
# Executes your `query` string:
c.execute(query)
matches = c.fetchall()
for i, match in enumerate(matches):
    print(i, "->", match)

0 -> ('Vuduc', 1.0)
1 -> ('Vuduc', 3.0)
2 -> ('Vuduc', 4.0)
3 -> ('Chau', 2.0)
4 -> ('Chau', 3.0)
5 -> ('Chau', 4.0)
6 -> ('Bader', None)
7 -> ('Sokol', 4.0)
8 -> ('Sokol', 4.0)
9 -> ('Rozga', None)
10 -> ('Zha', None)
11 -> ('Park', None)
12 -> ('Vetter', None)
13 -> ('Brown', None)


### Aggregations

In [25]:
query = '''
        SELECT gtid, AVG(grade)
        FROM Takes 
        GROUP BY gtid
'''

for match in c.execute(query):
    print(match)

(123, 2.6666666666666665)
(456, 3.0)
(991, 4.0)


### Cleanup

In [26]:
c.close()
conn.close()